In [286]:
from googlesearch import search
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import urllib
import os
import openai

In [287]:
a = search(' "Dental"+"CME"+"Course"', 
           advanced=True, sleep_interval=0.5, num_results=200)

# Get the title, url, and description; put them in a dataframe
raw_df = pd.DataFrame(columns=['title', 'url', 'description'])

for i, result in enumerate(a):
    raw_df.loc[i] = [result.title, result.url, result.description]

HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3D%252B%2522Dental%2522%252B%2522CME%2522%252B%2522COURSE%2522%26num%3D224%26hl%3Den%26start%3D278&hl=en&q=EgRJS-LEGKH7h6YGIjCilduBYAKdjaAMQFHYI1_4RORZ_iHTlWMZ0vh6GT7vlGl8z22bFa2Us_KGLBIdKOMyAXJaAUM

In [288]:
len(raw_df)

278

In [289]:
df = raw_df.copy()
df.head()

,title,url,description
0,Continuing Education | University of Illinois ...,https://dentistry.uic.edu/continuing-education/,Continuing education courses for dental profes...
1,Dental Continuing Education Courses,https://www.dentalcare.com/en-us/ce-courses,dentalcare.com covers all of your professional...
2,Continuing Education,https://www.ada.org/en/education/continuing-ed...,"The ADA provides cutting-edge, unique continui..."
3,DAOC Continuing Education Courses In Chicago,https://daoc.edu/continuing-education/,Continuing Education Courses. Register Online ...
4,Upcoming EventsIllinois Academy of General Den...,https://ilagd.org/events/,The purpose of the Academy is to serve the nee...


In [290]:
# For each url, get the main url and save it in a new column. Remove the http:// and www if they exist
# Get the domain name using urllib
df['main_url'] = df['url'].apply(lambda x: urllib.parse.urlparse(x).hostname)
# Get the domain extension
df['domain_ext'] = df['main_url'].apply(lambda x: x.split('.')[-1])

df.head(3)

,title,url,description,main_url,domain_ext
0,Continuing Education | University of Illinois ...,https://dentistry.uic.edu/continuing-education/,Continuing education courses for dental profes...,dentistry.uic.edu,edu
1,Dental Continuing Education Courses,https://www.dentalcare.com/en-us/ce-courses,dentalcare.com covers all of your professional...,www.dentalcare.com,com
2,Continuing Education,https://www.ada.org/en/education/continuing-ed...,"The ADA provides cutting-edge, unique continui...",www.ada.org,org


In [291]:
# Eliminate duplicates
df = df.drop_duplicates(subset=['main_url'], keep='first')

# eliminate rows with facebook, twitter, linkedin, youtube, instagram, pinterest, and wikipedia
df = df[~df['main_url'].str.contains('facebook')]
df = df[~df['main_url'].str.contains('twitter')]
df = df[~df['main_url'].str.contains('linkedin')]
df = df[~df['main_url'].str.contains('youtube')]
df = df[~df['main_url'].str.contains('instagram')]
df = df[~df['main_url'].str.contains('pinterest')]
df = df[~df['main_url'].str.contains('wikipedia')]
df = df[~df['main_url'].str.contains('google')]
df = df[~df['main_url'].str.contains('nih.gov')]
df = df[~df['main_url'].str.contains('doctor.com')]
df = df[~df['main_url'].str.contains('ebay')]
df = df[~df['main_url'].str.contains('amazon')]
df = df[~df['main_url'].str.contains('walmart')]
df = df[~df['main_url'].str.contains('craigslist')]
df = df[~df['main_url'].str.contains('webmd')]

print(len(df))
df.head(3)

267


,title,url,description,main_url,domain_ext
0,Continuing Education | University of Illinois ...,https://dentistry.uic.edu/continuing-education/,Continuing education courses for dental profes...,dentistry.uic.edu,edu
1,Dental Continuing Education Courses,https://www.dentalcare.com/en-us/ce-courses,dentalcare.com covers all of your professional...,www.dentalcare.com,com
2,Continuing Education,https://www.ada.org/en/education/continuing-ed...,"The ADA provides cutting-edge, unique continui...",www.ada.org,org


In [292]:
df["domain_ext"].value_counts()

com           112
edu            67
org            48
gov            16
in              8
net             3
uk              2
ca              2
education       1
de              1
pl              1
tz              1
fr              1
nl              1
mil             1
foundation      1
pk              1
Name: domain_ext, dtype: int64

In [160]:
# For each url, ping the main url and get the page title and description
# Use the User-Agent header to avoid getting blocked
count = 0
def get_title(url):
    global count
    count += 1
    print(count)
    
    try:
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, allow_redirects=True, timeout=10)
        soup = bs(r.text, 'html.parser')

        title, description = None, None
        try:
            title = soup.title.text
        except:
            pass
        try:
            description = soup.find("meta",  property="og:description")['content']
            
            if not description:
                description = soup.find("meta",  property="description")['content']
                
        except:
            pass

        return title, description
    except Exception as e:
        print(e)
        return (None, None)
    
results = (df['main_url'].apply(lambda x: get_title('http://' + x)))

1
2
3
4
5
6
7
8
9
10
11
12
13
HTTPSConnectionPool(host='www.3m.com', port=443): Read timed out. (read timeout=10)
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
HTTPSConnectionPool(host='ssc.dentalxp.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1125)')))
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
HTTPSConnectionPool(host='youngdental.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1125)')))
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
10

In [176]:
df['main_title'] = results.apply(lambda x: x[0])
df['main_description'] = results.apply(lambda x: x[1])

# Replace line breaks with spaces
df['main_title'] = df['main_title'].str.replace('\n', ' ')
df['main_description'] = df['main_description'].str.replace('\n', ' ')
df['description'] = df['description'].str.replace('\n', ' ')
df['title'] = df['title'].str.replace('\n', ' ')

# Replace all non-ascii characters with spaces
df['main_title'] = df['main_title'].str.encode('ascii', 'ignore').str.decode('ascii')
df['main_description'] = df['main_description'].str.encode('ascii', 'ignore').str.decode('ascii')
df['description'] = df['description'].str.encode('ascii', 'ignore').str.decode('ascii')
df['title'] = df['title'].str.encode('ascii', 'ignore').str.decode('ascii')

# Replace Just a moment...	with None
df['main_title'] = df['main_title'].apply(lambda x: None if x == 'Just a moment...' else x).str.strip()

In [177]:
df

,title,url,description,main_url,domain_ext,main_title,main_description,combined_description
0,Continuing Education | University of Illinois ...,https://dentistry.uic.edu/continuing-education/,Continuing education courses for dental profes...,dentistry.uic.edu,edu,College of Dentistry | University of Illinois ...,None,College of Dentistry | University of Illinois ...
1,Dental Continuing Education Courses,https://www.dentalcare.com/en-us/ce-courses,dentalcare.com covers all of your professional...,www.dentalcare.com,com,Dental Care Information for Professionals | De...,Dentalcare.com provides free continuing educat...,Dentalcare.com provides free continuing educat...
2,Continuing Education,https://www.ada.org/en/education/continuing-ed...,"The ADA provides cutting-edge, unique continui...",www.ada.org,org,Home | American Dental Association,The American Dental Association is your source...,The American Dental Association is your source...
3,DAOC Continuing Education Courses In Chicago,https://daoc.edu/continuing-education/,Continuing Education Courses. Register Online ...,daoc.edu,edu,Dental Assistant School In Chicago - DAOC: Den...,Your Oral Health Matters to Us Get Brighter Sm...,Your Oral Health Matters to Us Get Brighter Sm...
4,Dental Continuing Education Courses,https://www.cosmedent.com/continuing-education/,Cosmedent offers dental continuing education f...,www.cosmedent.com,com,Cosmetic Dental Supply & Education Company | C...,Cosmedent is your cosmetic dental supply compa...,Cosmedent is your cosmetic dental supply compa...
...,...,...,...,...,...,...,...,...
197,Trends in Dental CE Courses Shed Light on Indu...,https://www.oralhealthgroup.com/features/trend...,"Aug 31, 2018 — It's probably no surprise that ...",www.oralhealthgroup.com,com,Home - Oral Health Group,MARKETPLACE NEWS NEWS,MARKETPLACE NEWS NEWS
198,Overview & Guidelines - South Dakota Board of ...,https://www.sdboardofdentistry.org/continuingE...,"Jun 8, 2023 — Dentists must earn 100 hours of ...",www.sdboardofdentistry.org,org,Home - South Dakota Board of Dentistry,None,Home - South Dakota Board of Dentistry
199,Dental Hygiene CE Courses,https://healthprofessions.uams.edu/programs/de...,Completion fulfills the required continuing ed...,healthprofessions.uams.edu,edu,College of Health Professions,UAMS,UAMS
200,Continuing Education,https://www.vsds.org/meetings-events/continuin...,The Vermont State Dental Society is Vermont's ...,www.vsds.org,org,VSDS | Vermont State Dental Society,None,VSDS | Vermont State Dental Society


In [234]:
# create combined column for description:
# if main_description is null, use description
# if description is null, use main_title
# if main_title is null, use title
df['combined_description'] =  df['main_description'].fillna(df['description'].fillna(df['main_title'].fillna(df['title'])))

In [235]:
# export some columns to csv

title = 'dental_cme'

i = 0
while os.path.exists(title + str(i)+ '.csv'):
    i += 1


cols = ['title', 'url', 'description', 'main_url', 'main_title',
            'main_description', 'combined_description']

out_df = df[cols]
out_df.sort_values(by='main_url', inplace=True)

df.to_csv(title + str(i) + '.csv', columns=cols, index=False)



/var/folders/t9/_1b7nbbs1dj7crc4dq2d_mxc0000gn/T/ipykernel_18804/2286200015.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_df.sort_values(by='main_url', inplace=True)


In [283]:
# OpenAI API part to parse:
openai.api_key = os.getenv("OPENAI_API_KEY")

def open_ai_call(description):
  prompt = f"""You pull information out of blocks of info about companies: 
1. The company name (as short as possible) 
2. Whether or not the group provides dental continuing education courses, sometimes called CME, CE, or CDE 
3. A one sentence summary. 

Response format: Separate responses with "|||". Do not use linebreaks, single "|", or otherwise

Basic Example: Company Name ||| Offer CE Courses (True/False/Unknown) ||| Clear Description
Example: UIC ||| True ||| College of Dentistry at the University of Illinois Chicago, offering CE courses for dentists and dental hygienists.

Description: {description}
===
1.
"""

  response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
  )
  return response

In [246]:
tokens_used = 0

In [284]:
# Create a new dataframe, adding the parsed info
parsed_df = pd.DataFrame(columns=['ai_title', 'ai_offers', 'ai_description'])

# run the openai call on the first 10 rows
for i, row in df.iterrows():
    print(i)

    title = row['main_title']

    if not title:
        title = row['title']

    description = title + ' ' + row['description']

    response = open_ai_call(description)

    total_tokens = int(response["usage"]["total_tokens"])
    global tokens_used
    tokens_used += total_tokens

    split_response = response['choices'][0]['text'].split('|||')

    if len(split_response) == 1:
        split_response = response['choices'][0]['text'].split('|')

    title, offers, description = None, None, None

    if len(split_response) >= 3:
        title, offers, description = split_response[:3]
    if len(split_response) == 2:
        title, offers = split_response
        # print(response)
    if len(split_response) == 1:
        title = split_response[0]
        # print(response)

    parsed_df.loc[i] = [title, offers, description]

parsed_df

    

0


AuthenticationError: No API key provided. (HINT: set your API key in code using "openai.api_key = <API-KEY>", or you can set the environment variable OPENAI_API_KEY=<API-KEY>). You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details, or email support@openai.com if you have any questions.

In [296]:
parsed_df.head(2)

,ai_title,ai_offers,ai_description


In [280]:
# Join the two dataframes, keeping only the rows that have ai generated info
new_df = df.join(parsed_df, how='inner')
len(new_df)

196

In [295]:
new_df.head(2)

,title,url,description,main_url,domain_ext,main_title,main_description,combined_description,ai_title,ai_offers,ai_description
0,Continuing Education | University of Illinois ...,https://dentistry.uic.edu/continuing-education/,Continuing education courses for dental profes...,dentistry.uic.edu,edu,College of Dentistry | University of Illinois ...,None,Continuing education courses for dental profes...,UIC,True,University of Illinois Chicago College of Dent...
1,Dental Continuing Education Courses,https://www.dentalcare.com/en-us/ce-courses,dentalcare.com covers all of your professional...,www.dentalcare.com,com,Dental Care Information for Professionals | De...,Dentalcare.com provides free continuing educat...,Dentalcare.com provides free continuing educat...,Dentalcare.com,True,Conducting professional dental educational co...


In [258]:
new_df.sort_values(by='main_url', inplace=True)
new_df.sort_values(by='ai_offers', inplace=True, ascending=False)

In [259]:
new_df.head(3)

,title,url,description,main_url,domain_ext,main_title,main_description,combined_description,ai_title,ai_offers,ai_description
85,Online Dental Practice Continuing Education (C...,https://www.dentists-advantage.com/Prevention-...,The presentation will highlight common dental ...,www.dentists-advantage.com,com,Dental Malpractice Insurance for Dentists | De...,None,The presentation will highlight common dental ...,Dentist's Advantage,Unknown,An educational webinar presentation highlighti...
104,Continuing Education - School of Dentistry | S...,https://ce.dentistry.vcu.edu/,Dentist cleaning a child's teeth. Start learni...,ce.dentistry.vcu.edu,edu,Continuing Education - School of Dentistry | S...,None,Dentist cleaning a child's teeth. Start learni...,VCU School of Dentistry,True,Virginia Commonwealth University School of Den...
0,Continuing Education | University of Illinois ...,https://dentistry.uic.edu/continuing-education/,Continuing education courses for dental profes...,dentistry.uic.edu,edu,College of Dentistry | University of Illinois ...,None,Continuing education courses for dental profes...,UIC,True,University of Illinois Chicago College of Dent...


In [260]:
# Go through ai columns and replace "\n" with " "
new_df['ai_description'] = new_df['ai_description'].str.replace('\n', ' ')
new_df['ai_offers'] = new_df['ai_offers'].str.replace('\n', ' ')
new_df['ai_title'] = new_df['ai_title'].str.replace('\n', ' ')

In [294]:
new_df.head(2)

,title,url,description,main_url,domain_ext,main_title,main_description,combined_description,ai_title,ai_offers,ai_description
0,Continuing Education | University of Illinois ...,https://dentistry.uic.edu/continuing-education/,Continuing education courses for dental profes...,dentistry.uic.edu,edu,College of Dentistry | University of Illinois ...,None,Continuing education courses for dental profes...,UIC,True,University of Illinois Chicago College of Dent...
1,Dental Continuing Education Courses,https://www.dentalcare.com/en-us/ce-courses,dentalcare.com covers all of your professional...,www.dentalcare.com,com,Dental Care Information for Professionals | De...,Dentalcare.com provides free continuing educat...,Dentalcare.com provides free continuing educat...,Dentalcare.com,True,Conducting professional dental educational co...


In [257]:
new_df.columns

Index(['title', 'url', 'description', 'main_url', 'domain_ext', 'main_title',
       'main_description', 'combined_description', 'ai_title', 'ai_offers',
       'ai_description'],
      dtype='object')

In [274]:
export_columns = ['ai_title', 'ai_offers', 'ai_description', 'main_title', 
                  'url', 'main_url', 'description']

rename_dict = {'ai_title': 'Title', 
               'ai_offers': 'Group Offers?', 
               'ai_description': 'Description', 
               'url': 'CE Course URL'}

In [275]:
export_df = new_df[export_columns]

export_df.rename(columns=rename_dict, inplace=True)

/var/folders/t9/_1b7nbbs1dj7crc4dq2d_mxc0000gn/T/ipykernel_18804/1375993771.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export_df.rename(columns=rename_dict, inplace=True)


In [297]:
export_df.head(2)

,Title,Group Offers?,Description,main_title,CE Course URL,main_url,description
85,Dentist's Advantage,Unknown,An educational webinar presentation highlighti...,Dental Malpractice Insurance for Dentists | De...,https://www.dentists-advantage.com/Prevention-...,www.dentists-advantage.com,The presentation will highlight common dental ...
104,VCU School of Dentistry,True,Virginia Commonwealth University School of Den...,Continuing Education - School of Dentistry | S...,https://ce.dentistry.vcu.edu/,ce.dentistry.vcu.edu,Dentist cleaning a child's teeth. Start learni...


In [277]:
# export to csv
export_df.to_csv('dental_cme_parsed.csv', index=False)